In [ ]:
%run ../fyers_auth.ipynb
import sys
sys.path.append('../shared')
from data_fetching_func import dynamicDataFetching,clean_data 
import pandas as pd
import datetime as dt

In [ ]:
startDate = dt.date(2012,10,20)
endDate = dt.date.today()
symbol="NSE:SBIN-EQ"
resolution = "1D"
df = dynamicDataFetching(fyers,startDate,endDate,symbol,resolution)
print(df.dtypes)
print(df.describe())

In [ ]:
clean_data(df)

In [ ]:
# print(df.head())
df.shape

In [ ]:
df.head

In [ ]:
# Export df to csv
fileName = "5Y_SBIN_1min.csv"
# df.to_csv(f"../data/{fileName}")